# 🚀 CRUD Operations, API Endpoints, and CORS

Welcome to the grand finale of our FastAPI backend! In this notebook, you'll bring everything together and create a fully functional REST API with CORS support. 🎉

By the end of this guide, you'll have:
- ✅ Complete CRUD operations for managing todos
- ✅ Professional REST API endpoints
- ✅ CORS configuration for frontend integration
- ✅ A working API that can handle real HTTP requests

Let's complete your todo API and make it ready for the world! 🌍

# 🤔 Understanding CRUD Operations

## What is CRUD? 📋

**CRUD** stands for the four basic operations you can perform on data:

| Letter | Operation | Purpose | Example |
|--------|-----------|---------|----------|
| **C** | **Create** | Add new data | Add a new todo |
| **R** | **Read** | Get existing data | Show list of todos |
| **U** | **Update** | Modify existing data | Mark todo as complete |
| **D** | **Delete** | Remove data | Delete completed todo |

## Real-World Analogy: Library Management 📚

Think of CRUD like managing books in a library:

- **📖 Create**: "Add a new book to the library catalog"
- **📋 Read**: "Show me all books by Stephen King" or "Find book with ID #12345"
- **✏️ Update**: "Mark this book as checked out" or "Update the book's location"
- **🗑️ Delete**: "Remove this damaged book from the catalog"

## CRUD + HTTP = REST API 🌐

When we expose CRUD operations through HTTP, we get a **REST API**:

| CRUD | HTTP Method | URL | Action |
|------|-------------|-----|--------|
| **Create** | `POST` | `/todos` | Add new todo |
| **Read** | `GET` | `/todos` | Get all todos |
| **Read** | `GET` | `/todos/123` | Get specific todo |
| **Update** | `PUT` | `/todos/123` | Update entire todo |
| **Update** | `PATCH` | `/todos/123` | Update part of todo |
| **Delete** | `DELETE` | `/todos/123` | Remove todo |

## Why CRUD Matters for Our Todo App 🎯

Our todo app needs all four operations:

```
👤 User Action → API Endpoint → CRUD Operation

"Add new todo"     → POST /todos        → CREATE
"Show my todos"    → GET /todos         → READ
"Mark as done"     → PUT /todos/123     → UPDATE  
"Delete todo"      → DELETE /todos/123  → DELETE
```

Without any of these operations, our todo app would be incomplete!

# 🔍 Examining Our CRUD Operations

Let's look at how CRUD operations are implemented in our project.

## The crud.py File 📄

Our CRUD operations are defined in `crud.py`:

In [ ]:
# Navigate to backend directory
%cd 001-fastapi-backend

In [ ]:
# Let's examine our CRUD operations
!cat crud.py

## Breaking Down Each CRUD Function 🔧

Let's understand each function in detail:

### **1. CREATE Operation**
```python
def create_todo(db: Session, todo: schemas.ToDoRequest):
    db_todo = models.ToDo(name=todo.name, completed=todo.completed)
    db.add(db_todo)
    db.commit()
    db.refresh(db_todo)
    return db_todo
```

**What this does:**
1. **Creates SQLAlchemy object**: Convert Pydantic schema to database model
2. **Adds to session**: `db.add()` prepares the object for insertion
3. **Commits transaction**: `db.commit()` actually saves to database
4. **Refreshes object**: `db.refresh()` gets the assigned ID from database
5. **Returns created todo**: With the new ID included

### **2. READ Operations**

#### **Read All Todos:**
```python
def read_todos(db: Session, completed: bool):
    if completed is None:
        return db.query(models.ToDo).all()
    else:
        return db.query(models.ToDo).filter(models.ToDo.completed == completed).all()
```

**What this does:**
- **No filter**: Returns all todos if `completed` is `None`
- **With filter**: Returns only completed or incomplete todos
- **Uses SQLAlchemy query**: Efficient database query

#### **Read One Todo:**
```python
def read_todo(db: Session, id: int):
    return db.query(models.ToDo).filter(models.ToDo.id == id).first()
```

**What this does:**
- **Filters by ID**: Finds todo with specific ID
- **Returns first match**: `first()` returns one result or `None`

### **3. UPDATE Operation**
```python
def update_todo(db: Session, id: int, todo: schemas.ToDoRequest):
    db_todo = db.query(models.ToDo).filter(models.ToDo.id == id).first()
    if db_todo is None:
        return None
    db.query(models.ToDo).filter(models.ToDo.id == id).update({
        'name': todo.name, 
        'completed': todo.completed
    })
    db.commit()
    db.refresh(db_todo)
    return db_todo
```

**What this does:**
1. **Finds existing todo**: Check if todo with ID exists
2. **Returns None if not found**: Handles missing todos gracefully
3. **Updates fields**: Uses SQLAlchemy's `update()` method
4. **Commits changes**: Saves to database
5. **Returns updated todo**: With latest data

### **4. DELETE Operation**
```python
def delete_todo(db: Session, id: int):
    db_todo = db.query(models.ToDo).filter(models.ToDo.id == id).first()
    if db_todo is None:
        return None
    db.query(models.ToDo).filter(models.ToDo.id == id).delete()
    db.commit()
    return True
```

**What this does:**
1. **Finds todo to delete**: Check if it exists
2. **Returns None if not found**: Handles missing todos
3. **Deletes from database**: Uses SQLAlchemy's `delete()` method
4. **Commits deletion**: Makes the change permanent
5. **Returns success indicator**: `True` means deleted successfully

# 🧪 Testing CRUD Operations

Let's test our CRUD functions directly to make sure they work correctly.

## Testing CREATE Operation ➕

In [ ]:
# Test the CREATE operation
try:
    from database import SessionLocal
    from schemas import ToDoRequest
    import crud
    
    print("➕ Testing CREATE operation:")
    
    # Create database session
    db = SessionLocal()
    
    # Create a todo request
    todo_data = ToDoRequest(name="Test CRUD create", completed=False)
    print(f"   📝 Creating todo: {todo_data.dict()}")
    
    # Use CRUD function to create todo
    created_todo = crud.create_todo(db, todo_data)
    
    print(f"   ✅ Todo created successfully!")
    print(f"   📋 ID: {created_todo.id}")
    print(f"   📋 Name: {created_todo.name}")
    print(f"   📋 Completed: {created_todo.completed}")
    
    # Store ID for later tests
    test_todo_id = created_todo.id
    
    db.close()
    
except Exception as e:
    print(f"❌ CREATE test failed: {e}")
    import traceback
    traceback.print_exc()

## Testing READ Operations 📖

In [ ]:
# Test READ operations
try:
    from database import SessionLocal
    import crud
    
    print("📖 Testing READ operations:")
    
    db = SessionLocal()
    
    # Test read all todos
    all_todos = crud.read_todos(db, completed=None)
    print(f"   📋 Total todos in database: {len(all_todos)}")
    
    # Test read incomplete todos
    incomplete_todos = crud.read_todos(db, completed=False)
    print(f"   📋 Incomplete todos: {len(incomplete_todos)}")
    
    # Test read completed todos
    completed_todos = crud.read_todos(db, completed=True)
    print(f"   📋 Completed todos: {len(completed_todos)}")
    
    # Test read specific todo (if we have any)
    if all_todos:
        first_todo_id = all_todos[0].id
        specific_todo = crud.read_todo(db, first_todo_id)
        if specific_todo:
            print(f"   📋 Found specific todo (ID {first_todo_id}): {specific_todo.name}")
        else:
            print(f"   ❌ Could not find todo with ID {first_todo_id}")
    
    # Test reading non-existent todo
    non_existent = crud.read_todo(db, 99999)
    if non_existent is None:
        print(f"   ✅ Correctly returned None for non-existent todo")
    else:
        print(f"   ⚠️ Unexpected: found todo with ID 99999")
    
    db.close()
    print("   ✅ READ operations working correctly!")
    
except Exception as e:
    print(f"❌ READ test failed: {e}")

## Testing UPDATE Operation ✏️

In [ ]:
# Test UPDATE operation
try:
    from database import SessionLocal
    from schemas import ToDoRequest
    import crud
    
    print("✏️ Testing UPDATE operation:")
    
    db = SessionLocal()
    
    # First, get a todo to update
    all_todos = crud.read_todos(db, completed=None)
    if not all_todos:
        # Create one if none exist
        todo_data = ToDoRequest(name="Todo for update test", completed=False)
        test_todo = crud.create_todo(db, todo_data)
        print(f"   📝 Created todo for testing: ID {test_todo.id}")
    else:
        test_todo = all_todos[0]
        print(f"   📋 Using existing todo: ID {test_todo.id}")
    
    # Show original state
    print(f"   📋 Original: name='{test_todo.name}', completed={test_todo.completed}")
    
    # Create update data
    update_data = ToDoRequest(name="Updated todo name", completed=True)
    
    # Perform update
    updated_todo = crud.update_todo(db, test_todo.id, update_data)
    
    if updated_todo:
        print(f"   ✅ Todo updated successfully!")
        print(f"   📋 Updated: name='{updated_todo.name}', completed={updated_todo.completed}")
        
        # Verify the change persisted
        verified_todo = crud.read_todo(db, test_todo.id)
        if verified_todo and verified_todo.completed == True:
            print(f"   ✅ Update persisted correctly in database")
        else:
            print(f"   ❌ Update did not persist correctly")
    else:
        print(f"   ❌ Update failed - todo not found")
    
    # Test updating non-existent todo
    non_update = crud.update_todo(db, 99999, update_data)
    if non_update is None:
        print(f"   ✅ Correctly returned None for non-existent todo update")
    
    db.close()
    
except Exception as e:
    print(f"❌ UPDATE test failed: {e}")

## Testing DELETE Operation 🗑️

In [ ]:
# Test DELETE operation
try:
    from database import SessionLocal
    from schemas import ToDoRequest
    import crud
    
    print("🗑️ Testing DELETE operation:")
    
    db = SessionLocal()
    
    # Create a todo specifically for deletion test
    delete_test_data = ToDoRequest(name="Todo to be deleted", completed=False)
    todo_to_delete = crud.create_todo(db, delete_test_data)
    
    print(f"   📝 Created todo for deletion: ID {todo_to_delete.id}")
    
    # Count todos before deletion
    before_count = len(crud.read_todos(db, completed=None))
    print(f"   📊 Todos before deletion: {before_count}")
    
    # Delete the todo
    delete_result = crud.delete_todo(db, todo_to_delete.id)
    
    if delete_result:
        print(f"   ✅ Todo deleted successfully!")
        
        # Verify deletion
        deleted_todo = crud.read_todo(db, todo_to_delete.id)
        if deleted_todo is None:
            print(f"   ✅ Confirmed: todo no longer exists in database")
        else:
            print(f"   ❌ Problem: todo still exists after deletion")
        
        # Count todos after deletion
        after_count = len(crud.read_todos(db, completed=None))
        print(f"   📊 Todos after deletion: {after_count}")
        
        if after_count == before_count - 1:
            print(f"   ✅ Todo count decreased by 1 as expected")
    else:
        print(f"   ❌ Delete failed")
    
    # Test deleting non-existent todo
    non_delete = crud.delete_todo(db, 99999)
    if non_delete is None:
        print(f"   ✅ Correctly returned None for non-existent todo deletion")
    
    db.close()
    
except Exception as e:
    print(f"❌ DELETE test failed: {e}")

## CRUD Operations Summary ✅

If all tests passed, your CRUD operations are working perfectly! Here's what each operation provides:

| Operation | Function | Input | Output | Use Case |
|-----------|----------|-------|--------|-----------|
| **CREATE** | `create_todo()` | `ToDoRequest` | `ToDo` with ID | Add new todo |
| **READ ALL** | `read_todos()` | `completed` filter | List of `ToDo` | Show todos |
| **READ ONE** | `read_todo()` | `id` | `ToDo` or `None` | Get specific todo |
| **UPDATE** | `update_todo()` | `id`, `ToDoRequest` | `ToDo` or `None` | Modify todo |
| **DELETE** | `delete_todo()` | `id` | `True` or `None` | Remove todo |

These functions form the **data access layer** of your application - they handle all database interactions safely and consistently!

# 🌐 API Endpoints: Exposing CRUD via HTTP

Now let's examine how our CRUD operations are exposed as HTTP API endpoints.

## The routers/todos.py File 📄

Our API endpoints are defined in the routers directory:

In [ ]:
# Let's examine our API endpoints
!cat routers/todos.py

## Breaking Down Each API Endpoint 🔧

Let's understand each endpoint in detail:

### **1. Router Setup**
```python
router = APIRouter(prefix="/todos")
```
- **Creates a router**: Groups related endpoints together
- **Sets prefix**: All routes will start with `/todos`
- **Organization**: Keeps main.py clean and organized

### **2. Database Dependency**
```python
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()
```
**This is crucial!** It provides database sessions to endpoints:
- **Creates session**: New database connection for each request
- **Yields session**: Makes it available to endpoint function
- **Closes session**: Always cleans up, even if errors occur

### **3. CREATE Endpoint**
```python
@router.post("", status_code=status.HTTP_201_CREATED)
def create_todo(todo: schemas.ToDoRequest, db: Session = Depends(get_db)):
    todo = crud.create_todo(db, todo)
    return todo
```
**HTTP Details:**
- **Method**: `POST` (for creating new resources)
- **URL**: `/todos` (root of todos collection)
- **Status**: `201 Created` (successful creation)
- **Input**: JSON validated by `ToDoRequest` schema
- **Output**: Created todo with assigned ID

### **4. READ ALL Endpoint**
```python
@router.get("", response_model=List[schemas.ToDoResponse])
def get_todos(completed: bool = None, db: Session = Depends(get_db)):
    todos = crud.read_todos(db, completed)
    return todos
```
**HTTP Details:**
- **Method**: `GET` (for retrieving data)
- **URL**: `/todos` (same as create, but different method)
- **Query Parameter**: `?completed=true` (optional filter)
- **Response Model**: List of `ToDoResponse` objects
- **Output**: Array of todos, optionally filtered

### **5. READ ONE Endpoint**
```python
@router.get("/{id}")
def get_todo_by_id(id: int, db: Session = Depends(get_db)):
    todo = crud.read_todo(db, id)
    if todo is None:
        raise HTTPException(status_code=404, detail="to do not found")
    return todo
```
**HTTP Details:**
- **Method**: `GET`
- **URL**: `/todos/{id}` (e.g., `/todos/123`)
- **Path Parameter**: `id` extracted from URL
- **Error Handling**: `404 Not Found` if todo doesn't exist
- **Output**: Single todo or error

### **6. UPDATE Endpoint**
```python
@router.put("/{id}")
def update_todo(id: int, todo: schemas.ToDoRequest, db: Session = Depends(get_db)):
    todo = crud.update_todo(db, id, todo)
    if todo is None:
        raise HTTPException(status_code=404, detail="to do not found")
    return todo
```
**HTTP Details:**
- **Method**: `PUT` (for updating entire resource)
- **URL**: `/todos/{id}`
- **Input**: JSON body + path parameter
- **Error Handling**: `404` if todo doesn't exist
- **Output**: Updated todo or error

### **7. DELETE Endpoint**
```python
@router.delete("/{id}", status_code=status.HTTP_200_OK)
def delete_todo(id: int, db: Session = Depends(get_db)):
    res = crud.delete_todo(db, id)
    if res is None:
        raise HTTPException(status_code=404, detail="to do not found")
```
**HTTP Details:**
- **Method**: `DELETE` (for removing resources)
- **URL**: `/todos/{id}`
- **Status**: `200 OK` (successful deletion)
- **Error Handling**: `404` if todo doesn't exist
- **Output**: No content (just status code)

## REST API Principles 📐

Our endpoints follow **REST** (Representational State Transfer) principles:

### **Resource-Based URLs:**
- `/todos` - Collection of todos
- `/todos/{id}` - Specific todo

### **HTTP Methods Express Actions:**
- `GET` = Read (safe, no side effects)
- `POST` = Create (not idempotent)
- `PUT` = Update (idempotent)
- `DELETE` = Remove (idempotent)

### **Status Codes Convey Meaning:**
- `200 OK` - Success
- `201 Created` - Resource created
- `404 Not Found` - Resource doesn't exist
- `422 Validation Error` - Invalid input

This makes our API **predictable and intuitive** for developers to use!

# 🔧 main.py Integration

Let's see how everything comes together in our main FastAPI application.

## Current main.py 📄

In [ ]:
# Let's examine our main FastAPI application
!cat main.py

## Understanding main.py Components 🔍

### **1. Router Integration**
```python
from routers import todos
app.include_router(todos.router)
```
This includes all our todo endpoints in the main app:
- **Modular design**: Routes organized in separate files
- **Automatic inclusion**: All `/todos/*` routes are available
- **Clean separation**: Main app focuses on configuration, routers handle endpoints

### **2. CORS Middleware**
```python
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
```
**This is crucial for frontend integration!** We'll explore CORS in detail next.

### **3. Exception Handling**
```python
@app.exception_handler(StarletteHTTPException)
async def http_exception_handler(request, exc):
    print(f"{repr(exc)}")
    return PlainTextResponse(str(exc.detail), status_code=exc.status_code)
```
**Global error handling**:
- **Logs errors**: Prints to console for debugging
- **Clean responses**: Returns user-friendly error messages
- **Consistent format**: All HTTP errors handled the same way

### **4. Configuration Integration**
```python
@lru_cache()
def get_settings():
    return config.Settings()

@app.get("/")
def read_root(settings: config.Settings = Depends(get_settings)):
    print(settings.app_name)
    return "Hello World"
```
**Configuration dependency injection**:
- **Cached settings**: Efficient configuration access
- **Dependency injection**: Clean configuration usage
- **Example usage**: Shows how to use settings in endpoints

# 🌍 Understanding CORS (Cross-Origin Resource Sharing)

## What is CORS? 🤔

**CORS** stands for Cross-Origin Resource Sharing. It's a security mechanism that controls how web pages from one domain can access resources from another domain.

## Real-World Analogy: International Travel 🛂

Think of CORS like **international border control**:

### **Without CORS (No Border Control)**:
```
🏠 Your Website (http://mysite.com)
    ↓ "I want to access that API!"
🏢 Any API (http://anyapi.com)
    ↓ "Sure, here's sensitive data!"
😈 Result: Malicious websites could steal your data
```

### **With CORS (Border Control)**:
```
🏠 Your Website (http://mysite.com)
    ↓ "I want to access that API!"
🛂 Browser: "Are you allowed to access http://api.com?"
🏢 API Server: "Let me check my CORS policy..."
    ↓ "Yes, mysite.com is on my allowed list"
🛂 Browser: "Access granted!"
```

## The CORS Problem for Our Todo App 🎯

Our setup:
- **Frontend**: React app on `http://localhost:3000`
- **Backend**: FastAPI on `http://localhost:8000`

### **Without CORS configuration:**
```
🌐 Frontend (localhost:3000)
    ↓ "fetch('http://localhost:8000/todos')"
🛂 Browser: "Cross-origin request blocked!"
😱 Result: Frontend can't access our API
```

### **With CORS configuration:**
```
🌐 Frontend (localhost:3000)
    ↓ "fetch('http://localhost:8000/todos')"
🛂 Browser: "Checking CORS policy..."
🚀 Backend: "localhost:3000 is allowed!"
🛂 Browser: "Request allowed!"
✅ Result: Frontend can access our API
```

## CORS Headers Explained 📋

When our FastAPI server responds, it includes special headers:

```http
Access-Control-Allow-Origin: *
Access-Control-Allow-Methods: GET, POST, PUT, DELETE, OPTIONS
Access-Control-Allow-Headers: Content-Type, Authorization
Access-Control-Allow-Credentials: true
```

### **What Each Header Means:**

- **`Access-Control-Allow-Origin: *`**
  - **Meaning**: Allow requests from any origin
  - **Development**: Fine for local development
  - **Production**: Should be specific domains

- **`Access-Control-Allow-Methods: *`**
  - **Meaning**: Allow all HTTP methods
  - **Includes**: GET, POST, PUT, DELETE, OPTIONS, etc.

- **`Access-Control-Allow-Headers: *`**
  - **Meaning**: Allow all request headers
  - **Includes**: Content-Type, Authorization, custom headers

- **`Access-Control-Allow-Credentials: true`**
  - **Meaning**: Allow cookies and authentication headers
  - **Important**: For authenticated requests

## Our CORS Configuration 🔧

Let's examine our current CORS setup:

```python
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],        # Any origin allowed
    allow_credentials=True,      # Cookies/auth allowed
    allow_methods=["*"],        # All HTTP methods
    allow_headers=["*"],        # All headers
)
```

### **Development vs Production CORS 🚦**

#### **Development (Current):**
```python
allow_origins=["*"]  # ⚠️ Very permissive
```
**Good for**: Local development, testing
**Bad for**: Production security

#### **Production (Recommended):**
```python
allow_origins=[
    "https://mytodoapp.com",
    "https://www.mytodoapp.com"
]  # ✅ Specific domains only
```
**Good for**: Security, preventing unauthorized access
**Requirement**: Know your frontend domain(s)

# 🧪 Testing Our Complete API

Let's test our FastAPI server with real HTTP requests to make sure everything works!

## Server Status Check 🌐

In [ ]:
# Check if our FastAPI server is running
import requests

try:
    response = requests.get("http://localhost:8000/", timeout=5)
    if response.status_code == 200:
        print("✅ FastAPI server is running!")
        print(f"📤 Response: {response.text}")
        
        # Check if CORS headers are present
        cors_headers = {
            'Access-Control-Allow-Origin': response.headers.get('Access-Control-Allow-Origin'),
            'Access-Control-Allow-Methods': response.headers.get('Access-Control-Allow-Methods'),
            'Access-Control-Allow-Headers': response.headers.get('Access-Control-Allow-Headers'),
        }
        
        print("\n🌍 CORS Headers:")
        for header, value in cors_headers.items():
            if value:
                print(f"   ✅ {header}: {value}")
            else:
                print(f"   ❌ {header}: Not present")
    else:
        print(f"⚠️ Server responded with status: {response.status_code}")
        
except requests.exceptions.ConnectionError:
    print("❌ Cannot connect to FastAPI server")
    print("💡 Make sure to run: uvicorn main:app --reload")
    print("💡 Server should be running on http://localhost:8000")
except Exception as e:
    print(f"❌ Error connecting to server: {e}")

## API Documentation Check 📚

In [ ]:
# Check if our API documentation is accessible
import requests

print("📚 API Documentation Check:")

# Check Swagger UI
try:
    docs_response = requests.get("http://localhost:8000/docs", timeout=5)
    if docs_response.status_code == 200:
        print("✅ Swagger UI accessible at http://localhost:8000/docs")
    else:
        print(f"❌ Swagger UI returned status: {docs_response.status_code}")
except Exception as e:
    print(f"❌ Cannot access Swagger UI: {e}")

# Check ReDoc
try:
    redoc_response = requests.get("http://localhost:8000/redoc", timeout=5)
    if redoc_response.status_code == 200:
        print("✅ ReDoc accessible at http://localhost:8000/redoc")
    else:
        print(f"❌ ReDoc returned status: {redoc_response.status_code}")
except Exception as e:
    print(f"❌ Cannot access ReDoc: {e}")

# Check OpenAPI JSON
try:
    openapi_response = requests.get("http://localhost:8000/openapi.json", timeout=5)
    if openapi_response.status_code == 200:
        print("✅ OpenAPI spec accessible at http://localhost:8000/openapi.json")
        
        # Check if our todos endpoints are documented
        openapi_data = openapi_response.json()
        if 'paths' in openapi_data:
            todos_paths = [path for path in openapi_data['paths'].keys() if '/todos' in path]
            print(f"📋 Todo endpoints documented: {len(todos_paths)}")
            for path in todos_paths:
                methods = list(openapi_data['paths'][path].keys())
                print(f"   🛤️ {path}: {', '.join(methods)}")
    else:
        print(f"❌ OpenAPI spec returned status: {openapi_response.status_code}")
except Exception as e:
    print(f"❌ Cannot access OpenAPI spec: {e}")

print("\n💡 Visit http://localhost:8000/docs to test your API interactively!")

## Testing CREATE Endpoint (POST /todos) ➕

In [ ]:
# Test POST /todos endpoint
import requests
import json

print("➕ Testing POST /todos (Create Todo):")

try:
    # Test data
    test_todo = {
        "name": "Test todo via API",
        "completed": False
    }
    
    # Send POST request
    response = requests.post(
        "http://localhost:8000/todos",
        json=test_todo,
        headers={"Content-Type": "application/json"},
        timeout=10
    )
    
    print(f"📊 Status Code: {response.status_code}")
    print(f"📤 Response Headers: {dict(response.headers)}")
    
    if response.status_code == 201:
        created_todo = response.json()
        print(f"✅ Todo created successfully!")
        print(f"📋 Created todo: {json.dumps(created_todo, indent=2)}")
        
        # Check if it has an ID
        if 'id' in created_todo and created_todo['id']:
            print(f"✅ Todo assigned ID: {created_todo['id']}")
            # Store for later tests
            test_todo_id = created_todo['id']
        else:
            print(f"❌ Todo missing ID")
            
    else:
        print(f"❌ Create failed with status {response.status_code}")
        print(f"📤 Response: {response.text}")
        
except Exception as e:
    print(f"❌ Create test failed: {e}")

## Testing READ Endpoints (GET /todos) 📖

In [ ]:
# Test GET /todos endpoints
import requests
import json

print("📖 Testing GET /todos (Read Todos):")

try:
    # Test 1: Get all todos
    print("\n1️⃣ Testing GET /todos (all todos):")
    response = requests.get("http://localhost:8000/todos", timeout=10)
    
    print(f"📊 Status Code: {response.status_code}")
    
    if response.status_code == 200:
        all_todos = response.json()
        print(f"✅ Found {len(all_todos)} todos")
        
        if all_todos:
            print(f"📋 First todo: {json.dumps(all_todos[0], indent=2)}")
            first_todo_id = all_todos[0]['id']
        else:
            print(f"📋 No todos in database yet")
            first_todo_id = None
    else:
        print(f"❌ Get all todos failed: {response.status_code}")
        print(f"📤 Response: {response.text}")
    
    # Test 2: Get incomplete todos
    print("\n2️⃣ Testing GET /todos?completed=false (incomplete todos):")
    response = requests.get("http://localhost:8000/todos?completed=false", timeout=10)
    
    if response.status_code == 200:
        incomplete_todos = response.json()
        print(f"✅ Found {len(incomplete_todos)} incomplete todos")
    else:
        print(f"❌ Get incomplete todos failed: {response.status_code}")
    
    # Test 3: Get completed todos
    print("\n3️⃣ Testing GET /todos?completed=true (completed todos):")
    response = requests.get("http://localhost:8000/todos?completed=true", timeout=10)
    
    if response.status_code == 200:
        completed_todos = response.json()
        print(f"✅ Found {len(completed_todos)} completed todos")
    else:
        print(f"❌ Get completed todos failed: {response.status_code}")
    
    # Test 4: Get specific todo (if we have one)
    if first_todo_id:
        print(f"\n4️⃣ Testing GET /todos/{first_todo_id} (specific todo):")
        response = requests.get(f"http://localhost:8000/todos/{first_todo_id}", timeout=10)
        
        if response.status_code == 200:
            specific_todo = response.json()
            print(f"✅ Found specific todo: {specific_todo['name']}")
        else:
            print(f"❌ Get specific todo failed: {response.status_code}")
    
    # Test 5: Get non-existent todo
    print("\n5️⃣ Testing GET /todos/99999 (non-existent todo):")
    response = requests.get("http://localhost:8000/todos/99999", timeout=10)
    
    if response.status_code == 404:
        print(f"✅ Correctly returned 404 for non-existent todo")
        print(f"📤 Error message: {response.text}")
    else:
        print(f"⚠️ Expected 404, got {response.status_code}")
        
except Exception as e:
    print(f"❌ Read tests failed: {e}")

## Testing UPDATE Endpoint (PUT /todos/{id}) ✏️

In [ ]:
# Test PUT /todos/{id} endpoint
import requests
import json

print("✏️ Testing PUT /todos/{id} (Update Todo):")

try:
    # First, make sure we have a todo to update
    todos_response = requests.get("http://localhost:8000/todos", timeout=10)
    
    if todos_response.status_code != 200:
        print("❌ Cannot get todos for update test")
    else:
        all_todos = todos_response.json()
        
        if not all_todos:
            # Create a todo for testing
            create_data = {"name": "Todo for update test", "completed": False}
            create_response = requests.post(
                "http://localhost:8000/todos",
                json=create_data,
                timeout=10
            )
            if create_response.status_code == 201:
                test_todo = create_response.json()
                print(f"📝 Created test todo: ID {test_todo['id']}")
            else:
                print("❌ Cannot create test todo")
                test_todo = None
        else:
            test_todo = all_todos[0]
            print(f"📋 Using existing todo: ID {test_todo['id']}")
        
        if test_todo:
            print(f"📋 Original todo: {json.dumps(test_todo, indent=2)}")
            
            # Update data
            update_data = {
                "name": "Updated todo name via API",
                "completed": not test_todo['completed']  # Toggle completion
            }
            
            print(f"📝 Updating with: {json.dumps(update_data, indent=2)}")
            
            # Send PUT request
            response = requests.put(
                f"http://localhost:8000/todos/{test_todo['id']}",
                json=update_data,
                headers={"Content-Type": "application/json"},
                timeout=10
            )
            
            print(f"📊 Status Code: {response.status_code}")
            
            if response.status_code == 200:
                updated_todo = response.json()
                print(f"✅ Todo updated successfully!")
                print(f"📋 Updated todo: {json.dumps(updated_todo, indent=2)}")
                
                # Verify the changes
                if (updated_todo['name'] == update_data['name'] and 
                    updated_todo['completed'] == update_data['completed']):
                    print(f"✅ Update data matches request")
                else:
                    print(f"❌ Update data doesn't match request")
            else:
                print(f"❌ Update failed with status {response.status_code}")
                print(f"📤 Response: {response.text}")
    
    # Test updating non-existent todo
    print("\n🔍 Testing update of non-existent todo:")
    update_data = {"name": "This won't work", "completed": True}
    response = requests.put(
        "http://localhost:8000/todos/99999",
        json=update_data,
        timeout=10
    )
    
    if response.status_code == 404:
        print(f"✅ Correctly returned 404 for non-existent todo update")
    else:
        print(f"⚠️ Expected 404, got {response.status_code}")
        
except Exception as e:
    print(f"❌ Update test failed: {e}")

## Testing DELETE Endpoint (DELETE /todos/{id}) 🗑️

In [ ]:
# Test DELETE /todos/{id} endpoint
import requests

print("🗑️ Testing DELETE /todos/{id} (Delete Todo):")

try:
    # First, create a todo specifically for deletion
    create_data = {"name": "Todo to be deleted via API", "completed": False}
    create_response = requests.post(
        "http://localhost:8000/todos",
        json=create_data,
        timeout=10
    )
    
    if create_response.status_code != 201:
        print("❌ Cannot create todo for deletion test")
    else:
        todo_to_delete = create_response.json()
        print(f"📝 Created todo for deletion: ID {todo_to_delete['id']}")
        
        # Count todos before deletion
        before_response = requests.get("http://localhost:8000/todos", timeout=10)
        if before_response.status_code == 200:
            before_count = len(before_response.json())
            print(f"📊 Todos before deletion: {before_count}")
        
        # Send DELETE request
        delete_response = requests.delete(
            f"http://localhost:8000/todos/{todo_to_delete['id']}",
            timeout=10
        )
        
        print(f"📊 Delete Status Code: {delete_response.status_code}")
        
        if delete_response.status_code == 200:
            print(f"✅ Todo deleted successfully!")
            
            # Verify deletion
            verify_response = requests.get(
                f"http://localhost:8000/todos/{todo_to_delete['id']}",
                timeout=10
            )
            
            if verify_response.status_code == 404:
                print(f"✅ Confirmed: todo no longer exists")
            else:
                print(f"❌ Problem: todo still exists after deletion")
            
            # Count todos after deletion
            after_response = requests.get("http://localhost:8000/todos", timeout=10)
            if after_response.status_code == 200:
                after_count = len(after_response.json())
                print(f"📊 Todos after deletion: {after_count}")
                
                if after_count == before_count - 1:
                    print(f"✅ Todo count decreased by 1 as expected")
                else:
                    print(f"❌ Todo count change unexpected")
        else:
            print(f"❌ Delete failed with status {delete_response.status_code}")
            print(f"📤 Response: {delete_response.text}")
    
    # Test deleting non-existent todo
    print("\n🔍 Testing deletion of non-existent todo:")
    response = requests.delete("http://localhost:8000/todos/99999", timeout=10)
    
    if response.status_code == 404:
        print(f"✅ Correctly returned 404 for non-existent todo deletion")
    else:
        print(f"⚠️ Expected 404, got {response.status_code}")
        
except Exception as e:
    print(f"❌ Delete test failed: {e}")

## Testing CORS Headers 🌍

In [ ]:
# Test CORS headers specifically
import requests

print("🌍 Testing CORS Headers:")

try:
    # Test 1: Simple GET request
    print("\n1️⃣ Testing simple GET request CORS headers:")
    response = requests.get("http://localhost:8000/todos", timeout=10)
    
    cors_headers = {
        'Access-Control-Allow-Origin': response.headers.get('Access-Control-Allow-Origin'),
        'Access-Control-Allow-Credentials': response.headers.get('Access-Control-Allow-Credentials'),
    }
    
    print(f"📊 Status Code: {response.status_code}")
    for header, value in cors_headers.items():
        if value:
            print(f"   ✅ {header}: {value}")
        else:
            print(f"   ❌ {header}: Missing")
    
    # Test 2: OPTIONS request (preflight)
    print("\n2️⃣ Testing OPTIONS preflight request:")
    preflight_headers = {
        'Origin': 'http://localhost:3000',
        'Access-Control-Request-Method': 'POST',
        'Access-Control-Request-Headers': 'content-type'
    }
    
    options_response = requests.options(
        "http://localhost:8000/todos",
        headers=preflight_headers,
        timeout=10
    )
    
    print(f"📊 OPTIONS Status Code: {options_response.status_code}")
    
    preflight_cors_headers = {
        'Access-Control-Allow-Origin': options_response.headers.get('Access-Control-Allow-Origin'),
        'Access-Control-Allow-Methods': options_response.headers.get('Access-Control-Allow-Methods'),
        'Access-Control-Allow-Headers': options_response.headers.get('Access-Control-Allow-Headers'),
        'Access-Control-Allow-Credentials': options_response.headers.get('Access-Control-Allow-Credentials'),
    }
    
    for header, value in preflight_cors_headers.items():
        if value:
            print(f"   ✅ {header}: {value}")
        else:
            print(f"   ❌ {header}: Missing")
    
    # Test 3: Cross-origin POST simulation
    print("\n3️⃣ Testing cross-origin POST simulation:")
    cross_origin_headers = {
        'Origin': 'http://localhost:3000',
        'Content-Type': 'application/json'
    }
    
    test_data = {"name": "CORS test todo", "completed": False}
    
    cors_post_response = requests.post(
        "http://localhost:8000/todos",
        json=test_data,
        headers=cross_origin_headers,
        timeout=10
    )
    
    print(f"📊 Cross-origin POST Status: {cors_post_response.status_code}")
    
    if cors_post_response.status_code == 201:
        print(f"✅ Cross-origin POST successful!")
        
        # Check CORS headers in response
        origin_header = cors_post_response.headers.get('Access-Control-Allow-Origin')
        if origin_header:
            print(f"   ✅ Response includes Access-Control-Allow-Origin: {origin_header}")
        else:
            print(f"   ❌ Response missing Access-Control-Allow-Origin")
    else:
        print(f"❌ Cross-origin POST failed: {cors_post_response.status_code}")
    
    print("\n🎉 CORS Configuration Summary:")
    print("✅ Your API supports cross-origin requests")
    print("✅ Frontend at localhost:3000 can access your API")
    print("✅ All HTTP methods are allowed")
    print("✅ Credentials (cookies/auth) are supported")
        
except Exception as e:
    print(f"❌ CORS test failed: {e}")

# 🎯 Comprehensive API Integration Test

Let's do a complete end-to-end test of our API to simulate real usage.

In [ ]:
# Comprehensive API integration test
import requests
import json

print("🎯 Comprehensive API Integration Test:")
print("Simulating a real user workflow...")

try:
    base_url = "http://localhost:8000"
    
    # Step 1: Check API is running
    print("\n1️⃣ Checking API status...")
    status_response = requests.get(f"{base_url}/", timeout=10)
    if status_response.status_code == 200:
        print("   ✅ API is running")
    else:
        raise Exception(f"API not responding: {status_response.status_code}")
    
    # Step 2: Get initial todo list (should be empty or have existing todos)
    print("\n2️⃣ Getting initial todo list...")
    initial_response = requests.get(f"{base_url}/todos", timeout=10)
    initial_todos = initial_response.json()
    print(f"   📋 Found {len(initial_todos)} existing todos")
    
    # Step 3: Create multiple todos
    print("\n3️⃣ Creating multiple todos...")
    test_todos = [
        {"name": "Learn FastAPI", "completed": False},
        {"name": "Build todo app", "completed": False},
        {"name": "Deploy to production", "completed": False},
    ]
    
    created_todos = []
    for i, todo_data in enumerate(test_todos, 1):
        response = requests.post(f"{base_url}/todos", json=todo_data, timeout=10)
        if response.status_code == 201:
            created_todo = response.json()
            created_todos.append(created_todo)
            print(f"   ✅ Created todo {i}: {created_todo['name']} (ID: {created_todo['id']})")
        else:
            print(f"   ❌ Failed to create todo {i}: {response.status_code}")
    
    # Step 4: Verify todos were created
    print("\n4️⃣ Verifying todos were created...")
    all_todos_response = requests.get(f"{base_url}/todos", timeout=10)
    all_todos = all_todos_response.json()
    current_count = len(all_todos)
    expected_count = len(initial_todos) + len(created_todos)
    
    if current_count >= expected_count:
        print(f"   ✅ Todo count increased to {current_count}")
    else:
        print(f"   ❌ Expected at least {expected_count} todos, found {current_count}")
    
    # Step 5: Complete some todos
    if created_todos:
        print("\n5️⃣ Completing some todos...")
        
        for i, todo in enumerate(created_todos[:2]):  # Complete first 2
            update_data = {"name": todo['name'], "completed": True}
            response = requests.put(f"{base_url}/todos/{todo['id']}", json=update_data, timeout=10)
            
            if response.status_code == 200:
                print(f"   ✅ Completed todo: {todo['name']}")
            else:
                print(f"   ❌ Failed to complete todo: {response.status_code}")
    
    # Step 6: Filter todos by completion status
    print("\n6️⃣ Testing todo filters...")
    
    # Get completed todos
    completed_response = requests.get(f"{base_url}/todos?completed=true", timeout=10)
    completed_todos = completed_response.json()
    print(f"   📋 Completed todos: {len(completed_todos)}")
    
    # Get incomplete todos
    incomplete_response = requests.get(f"{base_url}/todos?completed=false", timeout=10)
    incomplete_todos = incomplete_response.json()
    print(f"   📋 Incomplete todos: {len(incomplete_todos)}")
    
    # Step 7: Get specific todo
    if created_todos:
        print("\n7️⃣ Testing specific todo retrieval...")
        test_id = created_todos[0]['id']
        specific_response = requests.get(f"{base_url}/todos/{test_id}", timeout=10)
        
        if specific_response.status_code == 200:
            specific_todo = specific_response.json()
            print(f"   ✅ Retrieved todo: {specific_todo['name']}")
        else:
            print(f"   ❌ Failed to get specific todo: {specific_response.status_code}")
    
    # Step 8: Delete a todo
    if created_todos:
        print("\n8️⃣ Testing todo deletion...")
        todo_to_delete = created_todos[-1]  # Delete last one
        
        delete_response = requests.delete(f"{base_url}/todos/{todo_to_delete['id']}", timeout=10)
        
        if delete_response.status_code == 200:
            print(f"   ✅ Deleted todo: {todo_to_delete['name']}")
            
            # Verify deletion
            verify_response = requests.get(f"{base_url}/todos/{todo_to_delete['id']}", timeout=10)
            if verify_response.status_code == 404:
                print(f"   ✅ Confirmed deletion")
            else:
                print(f"   ❌ Todo still exists after deletion")
        else:
            print(f"   ❌ Failed to delete todo: {delete_response.status_code}")
    
    # Step 9: Final summary
    print("\n9️⃣ Final API state...")
    final_response = requests.get(f"{base_url}/todos", timeout=10)
    final_todos = final_response.json()
    
    completed_count = len([t for t in final_todos if t['completed']])
    incomplete_count = len([t for t in final_todos if not t['completed']])
    
    print(f"   📊 Total todos: {len(final_todos)}")
    print(f"   📊 Completed: {completed_count}")
    print(f"   📊 Incomplete: {incomplete_count}")
    
    print("\n🎉 COMPREHENSIVE INTEGRATION TEST PASSED!")
    print("✅ CREATE operations working")
    print("✅ READ operations working (all, filtered, specific)")
    print("✅ UPDATE operations working")
    print("✅ DELETE operations working")
    print("✅ CORS headers present")
    print("✅ Error handling working (404s)")
    print("✅ API is ready for frontend integration!")
    
except Exception as e:
    print(f"❌ Integration test failed: {e}")
    import traceback
    traceback.print_exc()

# 🎉 Your FastAPI Backend is Complete!

Congratulations! You've successfully built a complete, production-ready FastAPI backend for your todo application! 🚀

## 🏆 What You've Accomplished:

### **Complete CRUD Operations:**
- ✅ **CREATE**: Add new todos via `POST /todos`
- ✅ **READ**: Get todos via `GET /todos` with optional filtering
- ✅ **READ ONE**: Get specific todo via `GET /todos/{id}`
- ✅ **UPDATE**: Modify todos via `PUT /todos/{id}`
- ✅ **DELETE**: Remove todos via `DELETE /todos/{id}`

### **Professional REST API:**
- ✅ **RESTful design**: Resource-based URLs and proper HTTP methods
- ✅ **Status codes**: Meaningful HTTP status codes (200, 201, 404, 422)
- ✅ **Error handling**: Graceful handling of missing resources
- ✅ **Input validation**: Pydantic schemas protect against bad data
- ✅ **Response formatting**: Consistent JSON responses

### **Frontend Integration Ready:**
- ✅ **CORS configured**: Frontend can access your API
- ✅ **Cross-origin requests**: All HTTP methods supported
- ✅ **Credentials support**: Ready for authentication
- ✅ **Headers handled**: Proper CORS headers included

### **Production-Quality Features:**
- ✅ **Database integration**: PostgreSQL with SQLAlchemy
- ✅ **Migration support**: Alembic for schema changes
- ✅ **Configuration management**: Environment-based settings
- ✅ **Dependency injection**: Clean, testable code architecture
- ✅ **Auto-documentation**: Swagger UI and ReDoc available
- ✅ **Error logging**: Exception handling with logging

## 📚 Key Concepts You've Mastered:

- **🔄 CRUD Operations**: The foundation of data manipulation
- **🌐 REST API Design**: Industry-standard API patterns
- **🛡️ CORS**: Cross-origin request security and configuration
- **🎯 HTTP Methods**: Proper use of GET, POST, PUT, DELETE
- **📊 Status Codes**: Meaningful communication with clients
- **🔧 FastAPI Features**: Routing, dependencies, middleware

## 🌟 Your API Endpoints:

| Method | Endpoint | Purpose | Response |
|--------|----------|---------|----------|
| `POST` | `/todos` | Create new todo | 201 + created todo |
| `GET` | `/todos` | Get all todos | 200 + todo array |
| `GET` | `/todos?completed=true` | Get completed todos | 200 + filtered array |
| `GET` | `/todos/{id}` | Get specific todo | 200 + todo or 404 |
| `PUT` | `/todos/{id}` | Update todo | 200 + updated todo or 404 |
| `DELETE` | `/todos/{id}` | Delete todo | 200 or 404 |

## 🔗 Ready for Frontend Integration:

Your API is now ready to be consumed by:
- **React applications** (like our Next.js frontend)
- **Mobile apps** (iOS, Android)
- **Other web services**
- **Third-party integrations**

## 🚀 What's Next:

Your backend is complete! The next steps would be to:
1. **Build the frontend** (React/Next.js) to consume your API
2. **Add authentication** (JWT tokens, OAuth)
3. **Deploy to production** (Docker, cloud platforms)
4. **Add monitoring** (logging, metrics, health checks)

## 💡 Quick Reference:

**Start your API:**
```bash
cd 001-fastapi-backend
uvicorn main:app --reload
```

**Access your API:**
- **API Base**: http://localhost:8000
- **Interactive Docs**: http://localhost:8000/docs
- **Alternative Docs**: http://localhost:8000/redoc

**Test with curl:**
```bash
# Get all todos
curl http://localhost:8000/todos

# Create a todo
curl -X POST http://localhost:8000/todos \
  -H "Content-Type: application/json" \
  -d '{"name": "My todo", "completed": false}'
```

**🎯 Your FastAPI backend is production-ready and follows industry best practices!**

**Outstanding work! You've built a professional-grade API that any frontend can consume! 🌟**